## PARIS SUBSIDIES: DATA JOINING

### LIBRAIRIES IMPORT

In [1]:
import pandas as pd
import numpy as np

### DATA IMPORT

#### Subsidies

In [2]:
subsidies = pd.read_feather('../00_DataFiles/02_Cleaned/ParisSubsidies_Records.feather')
subsidies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79968 entries, 0 to 79967
Data columns (total 8 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   Numéro de dossier                                79968 non-null  object
 1   Année budgétaire                                 79968 non-null  object
 2   Numéro Siret                                     79968 non-null  object
 3   Objet du dossier                                 79968 non-null  object
 4   Montant voté                                     79968 non-null  object
 5   Direction                                        79968 non-null  object
 6   Nature de la subvention                          79968 non-null  object
 7   Secteurs d'activités définies par l'association  79968 non-null  object
dtypes: object(8)
memory usage: 4.9+ MB


In [6]:
subsidies.isna().sum()

Numéro de dossier                                  0
Année budgétaire                                   0
Numéro Siret                                       0
Objet du dossier                                   0
Montant voté                                       0
Direction                                          0
Nature de la subvention                            0
Secteurs d'activités définies par l'association    0
dtype: int64

In [5]:
# Only some datatypes to correct
subsidies['Année budgétaire'] = subsidies['Année budgétaire'].astype(int)
subsidies['Montant voté'] = subsidies['Montant voté'].astype(int)

#### SIRENE

In [4]:
SIRENE = pd.read_feather('../00_DataFiles/02_Cleaned/ParisSubsidies_SIRENE.feather')
SIRENE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11108 entries, 0 to 11107
Data columns (total 13 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   categorieentreprise                 11108 non-null  object        
 1   geolocetablissement                 11108 non-null  object        
 2   trancheeffectifsunitelegale         11108 non-null  category      
 3   trancheeffectifsunitelegaletriable  11108 non-null  int64         
 4   siretsiegeunitelegale               11108 non-null  object        
 5   siret                               11108 non-null  object        
 6   etablissementsiege                  11108 non-null  int64         
 7   codepostaletablissement             11108 non-null  object        
 8   caractereemployeurunitelegale       11108 non-null  int64         
 9   datecreationunitelegale             11108 non-null  datetime64[ns]
 10  datecreationetablissem

In [7]:
SIRENE.isna().sum()

categorieentreprise                   0
geolocetablissement                   0
trancheeffectifsunitelegale           0
trancheeffectifsunitelegaletriable    0
siretsiegeunitelegale                 0
siret                                 0
etablissementsiege                    0
codepostaletablissement               0
caractereemployeurunitelegale         0
datecreationunitelegale               0
datecreationetablissement             0
denominationunitelegale               0
adresseetablissementcomplete          0
dtype: int64

In [29]:
# We just need to transform categoryentreprise into an ordered category
SIRENE.categorieentreprise = pd.Categorical(SIRENE.categorieentreprise, ['Non productif', 'PME', 'ETI', 'GE'], ordered = True)

### JOINING

In [30]:
joined = subsidies.join(SIRENE.set_index('siret'), on='Numéro Siret', how = 'left')
joined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79968 entries, 0 to 79967
Data columns (total 20 columns):
 #   Column                                           Non-Null Count  Dtype         
---  ------                                           --------------  -----         
 0   Numéro de dossier                                79968 non-null  object        
 1   Année budgétaire                                 79968 non-null  int64         
 2   Numéro Siret                                     79968 non-null  object        
 3   Objet du dossier                                 79968 non-null  object        
 4   Montant voté                                     79968 non-null  int64         
 5   Direction                                        79968 non-null  object        
 6   Nature de la subvention                          79968 non-null  object        
 7   Secteurs d'activités définies par l'association  79968 non-null  object        
 8   categorieentreprise                 

In [11]:
joined.isna().sum()

Numéro de dossier                                  0
Année budgétaire                                   0
Numéro Siret                                       0
Objet du dossier                                   0
Montant voté                                       0
Direction                                          0
Nature de la subvention                            0
Secteurs d'activités définies par l'association    0
categorieentreprise                                7
geolocetablissement                                7
trancheeffectifsunitelegale                        7
trancheeffectifsunitelegaletriable                 7
siretsiegeunitelegale                              7
etablissementsiege                                 7
codepostaletablissement                            7
caractereemployeurunitelegale                      7
datecreationunitelegale                            7
datecreationetablissement                          7
denominationunitelegale                       

In [13]:
joined.loc[joined['categorieentreprise'].isna()]

,Numéro de dossier,Année budgétaire,Numéro Siret,Objet du dossier,Montant voté,Direction,Nature de la subvention,Secteurs d'activités définies par l'association,categorieentreprise,geolocetablissement,trancheeffectifsunitelegale,trancheeffectifsunitelegaletriable,siretsiegeunitelegale,etablissementsiege,codepostaletablissement,caractereemployeurunitelegale,datecreationunitelegale,datecreationetablissement,denominationunitelegale,adresseetablissementcomplete
41308,2019_07630,2019,78426221400017,soutenir la jeune création artistique contempo...,2000,DAC,Projet,[Culture & Arts],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
42392,2019_07196,2019,51348142400036,Volontaires intervenant auprès de victimes d'a...,0,DASES,Non précisée,"[Défense des droits et des intérêts, Mémoire]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
57130,2019_05369,2019,51348142400036,Volontaires intervenant auprès de Victimes d'A...,10000,DPSP,Projet,"[Défense des droits et des intérêts, Mémoire]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
65774,2020_02265,2020,51348142400036,Volontaires intervenant auprès de victimes d'a...,10000,DPSP,Projet,"[Défense des droits et des intérêts, Mémoire]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
66014,2020_07211,2020,78426221400017,Soutenir la jeune création artistique contempo...,1500,DAC,Projet,[Culture & Arts],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
73185,2021_09363,2021,78426221400017,aide à la création et aux jeunes artistes,0,DAC,Non précisée,[Culture & Arts],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
74471,2021_06477,2021,12345677911345,Test-MS-22/12/20,0,DDCT,Non précisée,"[Communication & média, Social, Sport]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN


In [14]:
# Above lines are either tests or 'non diffusible', we can drop them.
df_drops = joined.loc[joined['categorieentreprise'].isna()]
df_drops.to_csv('../00_DataFiles/99_Dropped/ParisSubsidies_NoJoining.csv', index = False)
joined.dropna(inplace = True)

In [21]:
joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79961 entries, 0 to 79967
Data columns (total 20 columns):
 #   Column                                           Non-Null Count  Dtype         
---  ------                                           --------------  -----         
 0   Numéro de dossier                                79961 non-null  object        
 1   Année budgétaire                                 79961 non-null  int64         
 2   Numéro Siret                                     79961 non-null  object        
 3   Objet du dossier                                 79961 non-null  object        
 4   Montant voté                                     79961 non-null  int64         
 5   Direction                                        79961 non-null  object        
 6   Nature de la subvention                          79961 non-null  object        
 7   Secteurs d'activités définies par l'association  79961 non-null  object        
 8   categorieentreprise                 

### SAVE

#### CSV

In [17]:
joined.to_csv('../00_DataFiles/03_Joined/ParisSubsidies_Joined.csv', index = False)

#### Feather

In [23]:
joined.reset_index(drop=True).to_feather('../00_DataFiles/03_Joined/ParisSubsidies_Joined.feather')

In [28]:
joined['categorieentreprise'].unique()

array(['PME', 'ETI', 'Non productif', 'GE'], dtype=object)